In [2]:
import time
import pandas as pd
from selenium import webdriver
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException
from selenium.webdriver.common.by import By
df = pd.read_csv("kw_no_email_use_selenium.csv")

url_list = df['agent_url'].unique()
def get_driver():
    options = webdriver.FirefoxOptions()
    options.add_argument("--headless")
    options.add_argument("--no-sandbox")
    options.add_argument("--disable-dev-shm-usage")
    driver = webdriver.Firefox(options=options)
    return driver


def collect_agent_contact_details(driver):
    try:
        agent_info = driver.find_element_by_class_name("AgentInformation")
    except:
        agent_info = ""
    try:
        info_container = agent_info.find_elements_by_class_name("AgentInformation__factBody")
    except:
        info_container = ""
    try:
        agent_email = [email.text for email in info_container if email.get_attribute("aria-label") == 'Agent E-mail'][0]
    except:
        agent_email = ""
    try:
        mobile_number = driver.find_element_by_class_name("AgentInformation__phoneMobileNumber").text
    except:
        mobile_number = ""
    try:
        office_number = driver.find_element_by_class_name("AgentInformation__phoneOfficeNumber").text
    except:
        office_number = ""
    try:
        agent_website = [i.text for i in info_container if i.get_attribute("aria-label") == 'Agent Website'][0]
    except:
        agent_website = ""
    contact_dict = {"agent_email":agent_email,"mobile_number":mobile_number,"office_number":office_number,'agent_website':agent_website }
    return contact_dict


def get_agent_details(driver):
    try:
        agent_licenses = driver.find_element_by_class_name("AgentContent__licenses").text
    except:
        agent_licenses = ""
    try:
        agent_bio = driver.find_element_by_class_name("AgentContent__bio").text
    except:
        agent_bio = ""
    try:
        serviceAreas = driver.find_element_by_class_name("AgentContent__serviceAreas").text
    except:
        serviceAreas = ""
    try:
        agent_team_name = driver.find_element_by_class_name("AgentContent__teamName").text
    except:
        agent_team_name = ""
    try:
        agent_team_info_ = driver.find_element_by_class_name("AgentContent__teamText").text.split("\n")
        agent_team_info = ", ".join(agent_team_info_)
    except:
        agent_team_info = ""
    try:
        logo_url = driver.find_element_by_class_name('AgentContent__teamAvatar').find_element_by_class_name("KWImage__image").get_attribute("src")
    except:
        logo_url = ""
    agent_details = {"agent_licenses":agent_licenses, "agent_bio":agent_bio, "serviceAreas":serviceAreas,"agent_team_name":agent_team_name,
                    "agent_team_info":agent_team_info,'logo_url':logo_url}
    return agent_details


def social_media_details(driver):
    try:
        social_media_container = driver.find_element_by_class_name("AgentInformation__socialMedia").find_elements_by_class_name("link")
    except:
        social_media_container = []
    social_media_links = [social_media.get_attribute("href") for social_media in social_media_container]
    try:
        facebook = [facebook for facebook in social_media_links if 'facebook' in facebook][0]
    except:
        facebook = ""
    try:
        instagram = [instagram for instagram in social_media_links if 'instagram' in instagram][0]
    except:
        instagram = ""
    try:
        twitter = [twitter for twitter in social_media_links if 'twitter' in twitter][0]
    except:
        twitter = ""
    try:
        linkedin = [linkedin for linkedin in social_media_links if 'linkedin' in linkedin][0]
    except:
        linkedin = ""
    social_media_dict = {"facebook":facebook,"instagram":instagram,"twitter":twitter,"linkedin":linkedin}
    social_media_links_str = "|".join(social_media_links)
    return social_media_dict,social_media_links_str


def collect_other_info(driver):
    container = driver.find_elements_by_class_name("AgentContent__section")
    try:
        Market_Cente = [i for i in container if 'Market Center' in i.text][0].find_element_by_class_name("AgentContent__sectionText").text
    except:
        Market_Cente = ""
    try:
        Languages = [i for i in container if 'Languages' in i.text][0].find_element_by_class_name("AgentContent__sectionText").text
    except:
        Languages = ""
    try:
        Specialties_designation = [i for i in container if 'Specialties and Designations' in i.text][0].find_element_by_class_name("AgentContent__sectionText").text
    except:
        Specialties_designation = ""
    other_info = {"Market_Cente":Market_Cente, "Languages":Languages, "Specialties_Designations":Specialties_designation}
    return other_info

def wait_to_page_load(driver):
    try:
        w = WebDriverWait(driver, 3)
        w.until(EC.presence_of_element_located((By.CLASS_NAME,"AgentContent__name")))
        print("Page load happened")
    except Exception as e:
        print("Timeout happened no page load")
    return driver

url_count = 36000
driver_count = 0
driver = get_driver()
for url in url_list[url_count:37000]:
    print(url)
    if driver_count >=100:
        print("need to close driver")
        driver.quit()
        driver_count = 0
        driver = get_driver()
        print("new driver initialize****************")
    driver.get(url)
    driver = wait_to_page_load(driver)
    try:
        agent_name = driver.find_element_by_class_name("AgentContent__name").text
    except:
        agent_name = ""
    try:
        time.sleep(0.5)
        profile_url = driver.find_element_by_class_name("AvatarImage__bg").get_attribute("style").replace('background-image: url("','').replace('");','')
    except:
        profile_url = ""
    try:
        tag = driver.find_element_by_class_name("pill").text
    except:
        tag = ""
    try:
        role = driver.find_element_by_class_name("AgentContent__team").text
    except:
        role = ""
    try:
        location = driver.find_element_by_class_name("AgentContent__location").text
    except:
        location = ""
    contact_dict = collect_agent_contact_details(driver)
    agent_details = get_agent_details(driver)
    social_media_dict,social_media_links_str = social_media_details(driver)
    other_info = collect_other_info(driver)
    data_dict = {'agent_url':url,"agent_name":agent_name, 'agent_role':role,'location':location,"agent_dp":profile_url, "tag":tag,"agent_email":contact_dict['agent_email'], "mobile_number":contact_dict['mobile_number'],"office_number":contact_dict['office_number'],'agent_license':agent_details['agent_licenses'], 'agent_bio':agent_details['agent_bio'],'serviceAreas':agent_details['serviceAreas'],'office_name':agent_details['agent_team_name'],'office_address':agent_details['agent_team_info'],'logo_url':agent_details['logo_url'],'facebook':social_media_dict['facebook'], 'instagram':social_media_dict['instagram'],'twitter':social_media_dict['twitter'],'linkedin':social_media_dict['linkedin'],'Market_Cente':other_info['Market_Cente'],'agent_language':other_info['Languages'],'Specialties_Designations':other_info['Specialties_Designations'],'social_media_links_str':social_media_links_str,'agent_website':contact_dict['agent_website']}
    data_df = pd.DataFrame(data_dict,index=[0],columns=['agent_url','agent_name', 'agent_role','location','agent_dp', 'tag','agent_email', 'mobile_number','office_number', 'agent_license', 'agent_bio', 'serviceAreas','office_name', 'office_address','logo_url','facebook', 'instagram', 'twitter', 'linkedin','Market_Cente', 'agent_language', 'Specialties_Designations','social_media_links_str','agent_website'])
    with open("kw_data-emailSelenium_36k_37k.csv",'a',newline='',encoding='utf-8') as f:
        data_df.to_csv(f, mode='a',header=f.tell()==0)
    print(url_count,url)
    url_count+=1
    driver_count+=1
    print("***********************************")
driver.quit()


https://kw.com/agent/UPA-6725143252197208064-7
Timeout happened no page load
36000 https://kw.com/agent/UPA-6725143252197208064-7
***********************************
https://kw.com/agent/UPA-6587385177987125256-6
Page load happened
36001 https://kw.com/agent/UPA-6587385177987125256-6
***********************************
https://kw.com/agent/UPA-6587385201982402563-8
Page load happened
36002 https://kw.com/agent/UPA-6587385201982402563-8
***********************************
https://kw.com/agent/UPA-6587385079424073728-5
Page load happened
36003 https://kw.com/agent/UPA-6587385079424073728-5
***********************************
https://kw.com/agent/UPA-6587385109854515204-4
Page load happened
36004 https://kw.com/agent/UPA-6587385109854515204-4
***********************************
https://kw.com/agent/UPA-6587385044959744000-2
Page load happened
36005 https://kw.com/agent/UPA-6587385044959744000-2
***********************************
https://kw.com/agent/UPA-6592978207553126401-0
Page load ha

Page load happened
36053 https://kw.com/agent/UPA-6731906948000460800-6
***********************************
https://kw.com/agent/UPA-6587385266568421380-7
Page load happened
36054 https://kw.com/agent/UPA-6587385266568421380-7
***********************************
https://kw.com/agent/UPA-6587384970896707584-7
Page load happened
36055 https://kw.com/agent/UPA-6587384970896707584-7
***********************************
https://kw.com/agent/UPA-6587385180229976066-7
Page load happened
36056 https://kw.com/agent/UPA-6587385180229976066-7
***********************************
https://kw.com/agent/UPA-6587385070771077125-1
Page load happened
36057 https://kw.com/agent/UPA-6587385070771077125-1
***********************************
https://kw.com/agent/UPA-6587385356751454209-2
Page load happened
36058 https://kw.com/agent/UPA-6587385356751454209-2
***********************************
https://kw.com/agent/UPA-6780905704030486528-4
Page load happened
36059 https://kw.com/agent/UPA-6780905704030486528-

Page load happened
36106 https://kw.com/agent/UPA-6587385014124220420-1
***********************************
https://kw.com/agent/UPA-6587385341594824710-4
Page load happened
36107 https://kw.com/agent/UPA-6587385341594824710-4
***********************************
https://kw.com/agent/UPA-6672116857774235648-4
Page load happened
36108 https://kw.com/agent/UPA-6672116857774235648-4
***********************************
https://kw.com/agent/UPA-6587385237013565443-1
Page load happened
36109 https://kw.com/agent/UPA-6587385237013565443-1
***********************************
https://kw.com/agent/UPA-6587385174269722628-2
Page load happened
36110 https://kw.com/agent/UPA-6587385174269722628-2
***********************************
https://kw.com/agent/UPA-6638481443473080320-9
Page load happened
36111 https://kw.com/agent/UPA-6638481443473080320-9
***********************************
https://kw.com/agent/UPA-6587385455475253255-6
Page load happened
36112 https://kw.com/agent/UPA-6587385455475253255-

Page load happened
36159 https://kw.com/agent/UPA-6587385153621606405-1
***********************************
https://kw.com/agent/UPA-6755145956417654784-5
Page load happened
36160 https://kw.com/agent/UPA-6755145956417654784-5
***********************************
https://kw.com/agent/UPA-6587385388351635464-7
Timeout happened no page load
36161 https://kw.com/agent/UPA-6587385388351635464-7
***********************************
https://kw.com/agent/UPA-6587385151896748036-1
Page load happened
36162 https://kw.com/agent/UPA-6587385151896748036-1
***********************************
https://kw.com/agent/UPA-6785993159214088192-6
Page load happened
36163 https://kw.com/agent/UPA-6785993159214088192-6
***********************************
https://kw.com/agent/UPA-6587385353260298241-9
Page load happened
36164 https://kw.com/agent/UPA-6587385353260298241-9
***********************************
https://kw.com/agent/UPA-6587385190859345923-7
Page load happened
36165 https://kw.com/agent/UPA-658738519

Page load happened
36212 https://kw.com/agent/UPA-6587385174248751105-4
***********************************
https://kw.com/agent/UPA-6587385008373944324-5
Page load happened
36213 https://kw.com/agent/UPA-6587385008373944324-5
***********************************
https://kw.com/agent/UPA-6826882939557208064-2
Page load happened
36214 https://kw.com/agent/UPA-6826882939557208064-2
***********************************
https://kw.com/agent/UPA-6587385165438251009-9
Page load happened
36215 https://kw.com/agent/UPA-6587385165438251009-9
***********************************
https://kw.com/agent/UPA-6592268686090727424-3
Page load happened
36216 https://kw.com/agent/UPA-6592268686090727424-3
***********************************
https://kw.com/agent/UPA-6587385407159664640-9
Page load happened
36217 https://kw.com/agent/UPA-6587385407159664640-9
***********************************
https://kw.com/agent/UPA-6623279094206492674-5
Page load happened
36218 https://kw.com/agent/UPA-6623279094206492674-

...

Page load happened
36742 https://kw.com/agent/UPA-6587385260463583233-1
***********************************
https://kw.com/agent/UPA-6587385084525989891-9
Page load happened
36743 https://kw.com/agent/UPA-6587385084525989891-9
***********************************
https://kw.com/agent/UPA-6587385300977553416-3
Page load happened
36744 https://kw.com/agent/UPA-6587385300977553416-3
***********************************
https://kw.com/agent/UPA-6587385223434403845-1
Page load happened
36745 https://kw.com/agent/UPA-6587385223434403845-1
***********************************
https://kw.com/agent/UPA-6592234511177318404-4
Page load happened
36746 https://kw.com/agent/UPA-6592234511177318404-4
***********************************
https://kw.com/agent/UPA-6587385416714964999-8
Page load happened
36747 https://kw.com/agent/UPA-6587385416714964999-8
***********************************
https://kw.com/agent/UPA-6587385378590666756-8
Page load happened
36748 https://kw.com/agent/UPA-6587385378590666756-

Page load happened
36795 https://kw.com/agent/UPA-6587385170705154056-0
***********************************
https://kw.com/agent/UPA-6734852001168662528-8
Page load happened
36796 https://kw.com/agent/UPA-6734852001168662528-8
***********************************
https://kw.com/agent/UPA-6587385176533299201-6
Page load happened
36797 https://kw.com/agent/UPA-6587385176533299201-6
***********************************
https://kw.com/agent/UPA-6587385283356913669-2
Page load happened
36798 https://kw.com/agent/UPA-6587385283356913669-2
***********************************
https://kw.com/agent/UPA-6587385331204575235-2
Page load happened
36799 https://kw.com/agent/UPA-6587385331204575235-2
***********************************
https://kw.com/agent/UPA-6587385325506781186-4
need to close driver
new driver initialize****************
Page load happened
36800 https://kw.com/agent/UPA-6587385325506781186-4
***********************************
https://kw.com/agent/UPA-6587385357772410883-6
Page load h

Page load happened
36848 https://kw.com/agent/UPA-6587385028074688518-0
***********************************
https://kw.com/agent/UPA-6587385448170799107-1
Page load happened
36849 https://kw.com/agent/UPA-6587385448170799107-1
***********************************
https://kw.com/agent/UPA-6587385082915688450-5
Page load happened
36850 https://kw.com/agent/UPA-6587385082915688450-5
***********************************
https://kw.com/agent/UPA-6587385225788465152-5
Page load happened
36851 https://kw.com/agent/UPA-6587385225788465152-5
***********************************
https://kw.com/agent/UPA-6648979127133208576-8
Page load happened
36852 https://kw.com/agent/UPA-6648979127133208576-8
***********************************
https://kw.com/agent/UPA-6710285052729487360-9
Page load happened
36853 https://kw.com/agent/UPA-6710285052729487360-9
***********************************
https://kw.com/agent/UPA-6587385312441565189-6
Page load happened
36854 https://kw.com/agent/UPA-6587385312441565189-

Page load happened
36901 https://kw.com/agent/UPA-6835700530882031616-6
***********************************
https://kw.com/agent/UPA-6587385010578792450-7
Page load happened
36902 https://kw.com/agent/UPA-6587385010578792450-7
***********************************
https://kw.com/agent/UPA-6587385205781147650-5
Page load happened
36903 https://kw.com/agent/UPA-6587385205781147650-5
***********************************
https://kw.com/agent/UPA-6592947853504438272-0
Page load happened
36904 https://kw.com/agent/UPA-6592947853504438272-0
***********************************
https://kw.com/agent/UPA-6587385305594118149-2
Page load happened
36905 https://kw.com/agent/UPA-6587385305594118149-2
***********************************
https://kw.com/agent/UPA-6587384978176274437-6
Page load happened
36906 https://kw.com/agent/UPA-6587384978176274437-6
***********************************
https://kw.com/agent/UPA-6587384967089094658-6
Page load happened
36907 https://kw.com/agent/UPA-6587384967089094658-

Page load happened
36954 https://kw.com/agent/UPA-6715373866675998720-0
***********************************
https://kw.com/agent/UPA-6587385369507713029-5
Page load happened
36955 https://kw.com/agent/UPA-6587385369507713029-5
***********************************
https://kw.com/agent/UPA-6587384970447917056-2
Page load happened
36956 https://kw.com/agent/UPA-6587384970447917056-2
***********************************
https://kw.com/agent/UPA-6587385426029776902-4
Page load happened
36957 https://kw.com/agent/UPA-6587385426029776902-4
***********************************
https://kw.com/agent/UPA-6750116831896870912-4
Page load happened
36958 https://kw.com/agent/UPA-6750116831896870912-4
***********************************
https://kw.com/agent/UPA-6683407769431416832-6
Page load happened
36959 https://kw.com/agent/UPA-6683407769431416832-6
***********************************
https://kw.com/agent/UPA-6587384953132482565-3
Page load happened
36960 https://kw.com/agent/UPA-6587384953132482565-